# Embedding을 이용한 영화평 긍정/부정 예측 이진 분류 모델 


| **비교 항목**              | **OpenAI Embedding API**                          | **딥러닝 모델로 직접 임베딩 생성**                     |
|----------------------------|---------------------------------------------------|-------------------------------------------------------|
| **1. 개발 및 구현 난이도**   | ✅ **간단하고 빠름**: 이미 학습된 모델을 호출 | 🚧 **복잡함**: 모델 설계, 학습, 튜닝 필요 |
| **2. 초기 설정 비용**       | ✅ **거의 없음**: OpenAI 클라이언트 호출로 즉시 사용 가능 | 🚧 **높음**: 데이터 준비, 모델 학습, 하드웨어 필요 |
| **3. 모델 품질**            | ✅ **최신 기술**: OpenAI가 최적화된 사전 학습 모델 제공 | 품질이 데이터와 학습 과정에 크게 의존 |
| **4. 사용자 맞춤화**        | ❌ **제한적**: 제공된 모델을 변경하거나 재학습 불가 | ✅ **유연함**: 사용자의 데이터로 모델을 재학습 가능 |
| **8. 대규모 데이터 학습**    | ❌ **불가능**: OpenAI 모델은 이미 고정된 학습 데이터 기반 | ✅ **가능**: 특정 도메인 데이터를 학습해 성능 개선 가능 |
| **9. 도메인 적합성**         | ❌ **제한적**: OpenAI 모델이 일반화된 데이터에 최적화 | ✅ **높음**: 특정 도메인 데이터로 커스터마이징 가능 |
| **10. 유지보수와 지속성**    | ✅ **OpenAI가 관리**: API 호출만 하면 됨 | 🚧 **직접 관리**: 코드, 학습 데이터, 모델 업데이트 필요 |


In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv()) # read local .env file

True

In [2]:
from openai import OpenAI 
client = OpenAI()  # OpenAI 클라이언트 생성

embedding_model = "text-embedding-3-small"  # 사용할 텍스트 임베딩 모델

### Embedding을 특성으로 사용하여 Naver 영화평 예측 Machine Learning model 작성

In [3]:
import pandas as pd
import numpy as np

DATA_TRAIN_PATH = "https://raw.github.com/ironmanciti/Infran_NLP/main/data/naver_movie/ratings_train.txt"
DATA_TEST_PATH = "https://raw.github.com/ironmanciti/Infran_NLP/main/data/naver_movie/ratings_test.txt"

In [4]:
train_data = pd.read_csv(DATA_TRAIN_PATH, delimiter='\t')
print(train_data.shape)
train_data.head()

(150000, 3)


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [5]:
test_data = pd.read_csv(DATA_TEST_PATH, delimiter='\t')
print(test_data.shape)
test_data.head()

(50000, 3)


,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


- OpenAI API 가 유료 서비스이므로 학습 데이터 건수 축소

In [6]:
# 샘플링할 데이터 수
n_train_samples = 500
n_test_samples = 100

# 긍정 및 부정 리뷰 필터링 (학습 데이터)
train_positive = train_data[train_data['label'] == 1]
train_negative = train_data[train_data['label'] == 0]

# 긍정 및 부정 리뷰 필터링 (테스트 데이터)
test_positive = test_data[test_data['label'] == 1]
test_negative = test_data[test_data['label'] == 0]

# 긍정 및 부정 리뷰에서 각각 n_train_samples 수만큼 무작위 추출 (학습 데이터)
train_positive_sample = train_positive.sample(n=n_train_samples, random_state=42)
train_negative_sample = train_negative.sample(n=n_train_samples, random_state=42)

# 긍정 및 부정 리뷰에서 각각 n_test_samples 수만큼 무작위 추출 (테스트 데이터)
test_positive_sample = test_positive.sample(n=n_test_samples, random_state=42)
test_negative_sample = test_negative.sample(n=n_test_samples, random_state=42)

# 추출한 샘플 데이터 결합 (학습 데이터)
train_sample = pd.concat([train_positive_sample, train_negative_sample], ignore_index=True)

# 추출한 샘플 데이터 결합 (테스트 데이터)
test_sample = pd.concat([test_positive_sample, test_negative_sample], ignore_index=True)

# 학습 샘플 데이터 분포 출력
print("Train Sample Distribution:")
print(train_sample['label'].value_counts())

# 테스트 샘플 데이터 분포 출력
print("\nTest Sample Distribution:")
print(test_sample['label'].value_counts())

Train Sample Distribution:
label
1    500
0    500
Name: count, dtype: int64

Test Sample Distribution:
label
1    100
0    100
Name: count, dtype: int64


- 약 10분 소요 예상

In [7]:
%%time
# 텍스트 임베딩을 생성하는 함수 정의
def get_embedding(text, model):
    # 주어진 텍스트를 사용하여 임베딩 생성 요청을 보내고 결과를 반환
    return client.embeddings.create(input=text, model=model).data[0].embedding

# # 학습 데이터에 대한 임베딩 생성
# train_sample['embedding'] = train_sample.document.apply(lambda x: get_embedding(x, model=embedding_model))
# # 테스트 데이터에 대한 임베딩 생성
# test_sample['embedding'] = test_sample.document.apply(lambda x: get_embedding(x, model=embedding_model))

# # 임베딩이 포함된 데이터프레임을 CSV 파일로 저장
# train_sample.to_csv("output/train_sample.csv")
# test_sample.to_csv("output/test_sample.csv")

CPU times: total: 6.92 s
Wall time: 9min 58s


In [10]:
# 한번 저장해 놓은 임베딩을 시간과 비용 절약을 위해 재사용.
train_sample = pd.read_csv("output/train_sample.csv", index_col=0)
test_sample = pd.read_csv("output/test_sample.csv", index_col=0)

# 저장된 임베딩을 리스트 형태로 변환하여 다시 활용 가능하도록 변경
train_sample['embedding'] = train_sample.embedding.apply(eval).to_list()
test_sample['embedding'] = test_sample.embedding.apply(eval).to_list()

In [11]:
train_sample.head(2)

,id,document,label,embedding
0,5783305,운이가 살았으면 더 좋았을 꺼 같아요! 넘 재미있게 잘 봤습니다. 수고하셨어요,1,"[0.04455168917775154, -0.036102231591939926, -..."
1,4028446,스크림 표정 대박 ㅋㅋㅋㅋㅋㅋㅋㅋㅋ,1,"[0.018935009837150574, -0.02677968516945839, -..."


In [12]:
test_sample.head(2)

,id,document,label,embedding
0,4346958,Good!,1,"[0.008795172907412052, 0.0031343046575784683, ..."
1,9613528,"톰 행크스 영화는 믿고 보는데, 이 영화는 그 중에서도 정말 최고네요. 진심으로 추...",1,"[-0.045166924595832825, 0.038651760667562485, ..."


In [13]:
train_sample.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         1000 non-null   int64 
 1   document   1000 non-null   object
 2   label      1000 non-null   int64 
 3   embedding  1000 non-null   object
dtypes: int64(2), object(2)
memory usage: 39.1+ KB


## Machine Learning 학습을 위한 Text Feature Encoder로서의 Embedding 사용

In [14]:
# 리스트 형태의 임베딩을 numpy 배열로 변환
X_train = np.array(train_sample.embedding.tolist())
y_train = train_sample.label.values

X_test = np.array(test_sample.embedding.tolist())
y_test = test_sample.label.values

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1000, 1536), (200, 1536), (1000,), (200,))

### 이진 분류 모델 작성에 사용

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr = LogisticRegression()
lr.fit(X_train, y_train)

preds = lr.predict(X_test)

print(accuracy_score(y_test, preds))

0.735
